# 🟦 Python Context Managers  

Python **context managers** let you automatically run setup and cleanup
code using the `with` statement.


In [ ]:
with open("file.txt") as f:
    data = f.read()

display(data)

'some file'

In [1]:
display("File has been read successfully.")

'File has been read successfully.'

In [4]:
class LoggingContext:
    def __enter__(self):
        display("[ENTER] Starting block")
        return "You can use this value inside 'with'"

    def __exit__(self, exc_type, exc_value, exc_tb):
        if exc_type is None:
            display("[EXIT] Block finished successfully")
        else:
            display(f"[EXIT] Block raised: {exc_type.__name__}: {exc_value}")
        return False

# Example 3: Different types of exceptions
with LoggingContext():
    pass  # This will succeed

try:
    with LoggingContext():
        1 / 0  # ZeroDivisionError
except ZeroDivisionError:
    display("Division by zero handled outside context")

'[ENTER] Starting block'

'[EXIT] Block finished successfully'

'[ENTER] Starting block'

'[EXIT] Block raised: ZeroDivisionError: division by zero'

'Division by zero handled outside context'

In [5]:
class FileManager:
    def __init__(self, filename, mode):
        self.filename = filename
        self.mode = mode
        self.file = None

    def __enter__(self):
        display(f"Opening file: {self.filename}")
        self.file = open(self.filename, self.mode, encoding="utf-8")
        return self.file

    def __exit__(self, exc_type, exc_value, exc_tb):
        display(f"Closing file: {self.filename}")
        if self.file and not self.file.closed:
            self.file.close()
        return False

# Create a test file first
with open("test.txt", "w") as f:
    f.write("Hello, World!\nThis is a test file.\nLine 3 content.")

# Usage Example 1: Reading a file
with FileManager("test.txt", "r") as file:
    content = file.read()
    display("File content:")
    display(content)

# Usage Example 2: Writing to a file
with FileManager("output.txt", "w") as file:
    file.write("This is written using FileManager\n")
    file.write("Second line of text\n")

# Usage Example 3: Appending to a file
with FileManager("output.txt", "a") as file:
    file.write("This line was appended\n")

# Usage Example 4: Reading the file we just created
with FileManager("output.txt", "r") as file:
    content = file.read()
    display("Output file content:")
    display(content)

# Usage Example 5: Handling file that doesn't exist
try:
    with FileManager("nonexistent.txt", "r") as file:
        content = file.read()
except FileNotFoundError:
    display("File not found - handled gracefully")

'Opening file: test.txt'

'File content:'

'Hello, World!\nThis is a test file.\nLine 3 content.'

'Closing file: test.txt'

'Opening file: output.txt'

...

'Output file content:'

'This is written using FileManager\nSecond line of text\nThis line was appended\n'

'Closing file: output.txt'

'Opening file: nonexistent.txt'

'File not found - handled gracefully'

In [6]:
import os

class ChangeDirectory:
    def __init__(self, new_path):
        self.new_path = new_path
        self.old_path = None

    def __enter__(self):
        self.old_path = os.getcwd()
        os.chdir(self.new_path)
        display(f"Changed directory to {self.new_path}")
        return self.new_path

    def __exit__(self, exc_type, exc_value, exc_tb):
        os.chdir(self.old_path)
        display(f"Reverted directory to {self.old_path}")
        return False
    
# Usage examples for ChangeDirectory

# First, let's see our current directory
display(f"Current directory: {os.getcwd()}")

# Example 1: Change to /tmp directory
with ChangeDirectory("/tmp") as current_dir:
    display(f"Inside context: {os.getcwd()}")
    display(f"Returned value: {current_dir}")
    
    # Do some work in the new directory
    with open("temp_file.txt", "w") as f:
        f.write("This file is in /tmp")
    
    display("Created temp_file.txt")
    display(f"Files in current dir: {os.listdir('.')[:5]}")  # Show first 5 files

display(f"After context: {os.getcwd()}")

# Example 2: Nested directory changes
with ChangeDirectory("/tmp"):
    display("In /tmp")
    
    with ChangeDirectory("/"):
        display("In root directory")
        display(f"Files in root: {os.listdir('.')[:3]}")
    
    display("Back in /tmp")

display(f"Final directory: {os.getcwd()}")

# Example 3: Handling directory that doesn't exist
try:
    with ChangeDirectory("/nonexistent/path"):
        display("This won't be reached")
except FileNotFoundError:
    display("Directory doesn't exist - handled gracefully")

display(f"Still in original directory: {os.getcwd()}")

'Current directory: /opt/adam/common-lessons'

'Changed directory to /tmp'

'Inside context: /tmp'

'Returned value: /tmp'

'Created temp_file.txt'

...

'Back in /tmp'

'Reverted directory to /opt/adam/common-lessons'

'Final directory: /opt/adam/common-lessons'

"Directory doesn't exist - handled gracefully"

'Still in original directory: /opt/adam/common-lessons'

In [7]:
class ListTransaction:
    def __init__(self, target_list):
        self.target_list = target_list
        self._backup = None

    def __enter__(self):
        self._backup = self.target_list.copy()
        return self.target_list

    def __exit__(self, exc_type, exc_value, exc_tb):
        if exc_type is not None:
            display("Error occurred, rolling back list changes")
            self.target_list.clear()
            self.target_list.extend(self._backup)
        else:
            display("No error, keeping list changes")
        return False

# Usage examples for ListTransaction

# Example 1: Successful transaction (no rollback)
my_list = [1, 2, 3]
display(f"Original list: {my_list}")

with ListTransaction(my_list) as lst:
    lst.append(4)
    lst.append(5)
    lst.extend([6, 7])
    display(f"Modified list inside context: {lst}")

display(f"Final list after successful transaction: {my_list}")

# Example 2: Failed transaction (with rollback)
my_list = [1, 2, 3]
display(f"Original list: {my_list}")

try:
    with ListTransaction(my_list) as lst:
        lst.append(4)
        lst.append(5)
        display(f"Modified list before error: {lst}")
        raise ValueError("Something went wrong!")
        lst.append(6)  # This won't execute
except ValueError as e:
    display(f"Caught exception: {e}")

display(f"List after failed transaction (rolled back): {my_list}")

# Example 3: More complex operations
shopping_cart = ["apples", "bananas"]
display(f"Shopping cart: {shopping_cart}")

try:
    with ListTransaction(shopping_cart) as cart:
        cart.append("oranges")
        cart.append("milk")
        cart.remove("bananas")
        display(f"Cart during transaction: {cart}")
        
        # Simulate payment failure
        if len(cart) > 3:
            raise RuntimeError("Payment failed!")
        
except RuntimeError as e:
    display(f"Transaction failed: {e}")

display(f"Shopping cart after rollback: {shopping_cart}")

# Example 4: Successful complex transaction
inventory = ["widget1", "widget2", "widget3"]
display(f"Inventory: {inventory}")

with ListTransaction(inventory) as inv:
    inv.remove("widget2")  # Sold
    inv.append("widget4")  # Restocked
    inv.append("widget5")  # New item
    display(f"Updated inventory: {inv}")

display(f"Final inventory: {inventory}")

'Original list: [1, 2, 3]'

'Modified list inside context: [1, 2, 3, 4, 5, 6, 7]'

'No error, keeping list changes'

'Final list after successful transaction: [1, 2, 3, 4, 5, 6, 7]'

'Original list: [1, 2, 3]'

...

"Shopping cart after rollback: ['apples', 'oranges', 'milk']"

"Inventory: ['widget1', 'widget2', 'widget3']"

"Updated inventory: ['widget1', 'widget3', 'widget4', 'widget5']"

'No error, keeping list changes'

"Final inventory: ['widget1', 'widget3', 'widget4', 'widget5']"

In [8]:
class Suppress:
    def __init__(self, *exception_types):
        self.exception_types = exception_types

    def __enter__(self):
        return None

    def __exit__(self, exc_type, exc_value, exc_tb):
        if exc_type is None:
            return False
        if issubclass(exc_type, self.exception_types):
            display(f"Suppressed {exc_type.__name__}: {exc_value}")
            return True
        return False
# Usage examples for Suppress

# Example 1: Suppress ValueError
display("Example 1: Suppressing ValueError")
with Suppress(ValueError):
    display("About to raise ValueError...")
    int("not_a_number")  # This will raise ValueError
    display("This line won't execute")

display("Continued after suppressed ValueError")

# Example 2: Suppress multiple exception types
display("\nExample 2: Suppressing multiple exception types")
with Suppress(ValueError, TypeError):
    display("About to raise TypeError...")
    "string" + 5  # This will raise TypeError
    display("This line won't execute")

display("Continued after suppressed TypeError")

# Example 3: Exception not suppressed
display("\nExample 3: Exception NOT suppressed")
try:
    with Suppress(ValueError):
        display("About to raise ZeroDivisionError...")
        1 / 0  # This will raise ZeroDivisionError (not suppressed)
        display("This line won't execute")
except ZeroDivisionError as e:
    display(f"ZeroDivisionError was NOT suppressed: {e}")

# Example 4: No exception raised
display("\nExample 4: No exception raised")
with Suppress(ValueError, TypeError):
    display("No exception will be raised here")
    result = 10 * 5
    display(f"Result: {result}")

display("Block completed successfully")

# Example 5: Practical use case - ignoring file operations
display("\nExample 5: Practical use - ignoring file not found")
files_to_process = ["existing_file.txt", "missing_file.txt", "another_missing.txt"]

for filename in files_to_process:
    with Suppress(FileNotFoundError):
        with open(filename, 'r') as f:
            content = f.read()
            display(f"Successfully read {filename}")
    display(f"Finished processing {filename} (or skipped if missing)")

'Example 1: Suppressing ValueError'

'About to raise ValueError...'

"Suppressed ValueError: invalid literal for int() with base 10: 'not_a_number'"

'Continued after suppressed ValueError'

'\nExample 2: Suppressing multiple exception types'

...

'Finished processing existing_file.txt (or skipped if missing)'

"Suppressed FileNotFoundError: [Errno 2] No such file or directory: 'missing_file.txt'"

'Finished processing missing_file.txt (or skipped if missing)'

"Suppressed FileNotFoundError: [Errno 2] No such file or directory: 'another_missing.txt'"

'Finished processing another_missing.txt (or skipped if missing)'

In [10]:
from contextlib import contextmanager

@contextmanager
def my_context():
    display("Entering")
    try:
        yield "Value inside with"
    finally:
        display("Exiting")

# Usage examples for my_context function-based context manager

# Example 1: Normal usage
display("Example 1: Normal usage")
with my_context() as value:
    display(f"Inside context: {value}")
    display("Doing some work...")

# Example 2: With exception
display("\nExample 2: With exception")
try:
    with my_context() as value:
        display(f"Inside context: {value}")
        raise ValueError("Something went wrong!")
        display("This won't execute")
except ValueError as e:
    display(f"Caught exception: {e}")

# Example 3: Multiple uses
display("\nExample 3: Multiple uses")
for i in range(3):
    with my_context() as value:
        display(f"Iteration {i}: {value}")

# Example 4: Nested context managers
display("\nExample 4: Nested contexts")
with my_context() as outer:
    display(f"Outer context: {outer}")
    with my_context() as inner:
        display(f"Inner context: {inner}")
        display("Both contexts are active")

'Example 1: Normal usage'

'Entering'

'Inside context: Value inside with'

'Doing some work...'

'Exiting'

...

'Entering'

'Inner context: Value inside with'

'Both contexts are active'

'Exiting'

'Exiting'

In [11]:
import os
from contextlib import contextmanager

@contextmanager
def temp_env_var(key, value):
    old_value = os.environ.get(key)
    os.environ[key] = value
    try:
        yield
    finally:
        if old_value is None:
            del os.environ[key]
        else:
            os.environ[key] = old_value
            
# Usage examples for temp_env_var

# Example 1: Setting a new environment variable temporarily
display("Example 1: Setting a new environment variable")
display(f"Before: DEBUG = {os.environ.get('DEBUG', 'Not set')}")

with temp_env_var('DEBUG', 'True'):
    display(f"Inside context: DEBUG = {os.environ.get('DEBUG')}")
    display("Running some debug code...")

display(f"After: DEBUG = {os.environ.get('DEBUG', 'Not set')}")

# Example 2: Temporarily changing an existing environment variable
display("\nExample 2: Changing existing environment variable")
# Set an initial value
os.environ['MY_APP_MODE'] = 'production'
display(f"Initial: MY_APP_MODE = {os.environ.get('MY_APP_MODE')}")

with temp_env_var('MY_APP_MODE', 'testing'):
    display(f"Inside context: MY_APP_MODE = {os.environ.get('MY_APP_MODE')}")
    display("Running tests...")

display(f"Restored: MY_APP_MODE = {os.environ.get('MY_APP_MODE')}")

# Example 3: Multiple temporary environment variables
display("\nExample 3: Multiple temporary variables")
with temp_env_var('API_URL', 'https://test-api.example.com'):
    with temp_env_var('API_KEY', 'test-key-12345'):
        display(f"API_URL = {os.environ.get('API_URL')}")
        display(f"API_KEY = {os.environ.get('API_KEY')}")
        display("Making API calls with test configuration...")

display(f"After: API_URL = {os.environ.get('API_URL', 'Not set')}")
display(f"After: API_KEY = {os.environ.get('API_KEY', 'Not set')}")

# Example 4: With exception handling
display("\nExample 4: With exception")
try:
    with temp_env_var('TEMP_VAR', 'temporary_value'):
        display(f"TEMP_VAR = {os.environ.get('TEMP_VAR')}")
        raise ValueError("Something went wrong!")
except ValueError as e:
    display(f"Exception caught: {e}")

display(f"After exception: TEMP_VAR = {os.environ.get('TEMP_VAR', 'Not set')}")

# Example 5: Practical use case - testing with different configurations
display("\nExample 5: Testing different database configurations")
configs = [
    ('DB_HOST', 'localhost'),
    ('DB_PORT', '5432'),
    ('DB_NAME', 'test_db')
]

for key, value in configs:
    with temp_env_var(key, value):
        display(f"Testing with {key}={value}")
        # Simulate database connection test
        display(f"Connected to {os.environ.get('DB_HOST')}:{os.environ.get('DB_PORT')}/{os.environ.get('DB_NAME')}")
    
    display(f"Reset: {key} = {os.environ.get(key, 'Not set')}")

'Example 1: Setting a new environment variable'

'Before: DEBUG = Not set'

'Inside context: DEBUG = True'

'Running some debug code...'

'After: DEBUG = Not set'

...

'Connected to None:5432/None'

'Reset: DB_PORT = Not set'

'Testing with DB_NAME=test_db'

'Connected to None:None/test_db'

'Reset: DB_NAME = Not set'

In [ ]:
# Create test files first
with open("a.txt", "w") as f:
    f.write("Content of file A\nLine 2 of A\nLine 3 of A")

with open("b.txt", "w") as f:
    f.write("Content of file B\nLine 2 of B\nLine 3 of B")

# Example 1: Reading from both files simultaneously
with open("a.txt") as f1, open("b.txt") as f2:
    content_a = f1.read()
    content_b = f2.read()

    display("Content of a.txt:")
    display(content_a)
    display("\nContent of b.txt:")
    display(content_b)

# Example 2: Comparing files line by line
with open("a.txt") as f1, open("b.txt") as f2:
    lines_a = f1.readlines()
    lines_b = f2.readlines()

    display("Line-by-line comparison:")
    max_lines = max(len(lines_a), len(lines_b))

    for i in range(max_lines):
        line_a = lines_a[i].strip() if i < len(lines_a) else "[EOF]"
        line_b = lines_b[i].strip() if i < len(lines_b) else "[EOF]"

        display(f"Line {i + 1}:")
        display(f"  a.txt: {line_a}")
        display(f"  b.txt: {line_b}")
        display(f"  Match: {line_a == line_b}")

# Example 3: Copying from one file to another
with open("a.txt") as source, open("copy_of_a.txt", "w") as dest:
    for line in source:
        dest.write(f"[COPIED] {line}")

    display("Copied a.txt to copy_of_a.txt with prefix")

# Verify the copy
with open("copy_of_a.txt") as f:
    display("Copy content:")
    display(f.read())

# Example 4: Merging two files
with open("a.txt") as f1, open("b.txt") as f2, open("merged.txt", "w") as merged:
    merged.write("=== Content from a.txt ===\n")
    merged.write(f1.read())
    merged.write("\n=== Content from b.txt ===\n")
    merged.write(f2.read())

    display("Merged both files into merged.txt")

# Show the merged result
with open("merged.txt") as f:
    display("Merged file content:")
    display(f.read())

'Content of a.txt:'

'Content of file A\nLine 2 of A\nLine 3 of A'

'\nContent of b.txt:'

'Content of file B\nLine 2 of B\nLine 3 of B'

'Line-by-line comparison:'

...

'Copy content:'

'[COPIED] Content of file A\n[COPIED] Line 2 of A\n[COPIED] Line 3 of A'

'Merged both files into merged.txt'

'Merged file content:'

'=== Content from a.txt ===\nContent of file A\nLine 2 of A\nLine 3 of A\n=== Content from b.txt ===\nContent of file B\nLine 2 of B\nLine 3 of B'

In [ ]:
### Nested:
import time
from contextlib import contextmanager


@contextmanager
def timer(label="Block"):
    start = time.time()
    display(f"[{label}] Started")
    try:
        yield
    finally:
        end = time.time()
        display(f"[{label}] Finished in {end - start:.4f} seconds")


# Now use the nested context managers
with ChangeDirectory("/tmp"):
    with timer("inner"):
        # Do some work in the /tmp directory while timing it
        display(f"Current directory: {os.getcwd()}")

        # Create some files
        with open("temp_file1.txt", "w") as f:
            f.write("This is a temporary file")

        with open("temp_file2.txt", "w") as f:
            f.write("Another temporary file")

        # List files in the directory
        files = os.listdir(".")
        temp_files = [f for f in files if f.startswith("temp_file")]
        display(f"Files created: {temp_files}")

        # Simulate some work with a small delay
        time.sleep(0.1)

        # Read back one of the files
        with open("temp_file1.txt", "r") as f:
            content = f.read()
            display(f"File content: {content}")

'Changed directory to /tmp'

'[inner] Started'

'Current directory: /tmp'

"Files created: ['temp_file1.txt', 'temp_file2.txt', 'temp_file.txt']"

'File content: This is a temporary file'

'[inner] Finished in 0.1058 seconds'

'Reverted directory to /opt/adam/common-lessons'